# Personal Project main notebook

In [2]:
# Ignoring warning messages from python
import warnings
warnings.filterwarnings('ignore')

# General use imports
import pandas as pd
import numpy as np

# Visualization imports
import matplotlib.pyplot as plt
import seaborn as sns

# Modules and data
import requests
import acquire
from vega_datasets import data
from datetime import datetime

In [4]:
transactions = acquire.get_local_transactions()
transactions.head()

,Date,Description,Original Description,Amount,Transaction Type,Category,Account Name,Labels,Notes
0,6/14/2022,VARSITY TUTORS L DIRDEPDD ***********8777,VARSITY TUTORS L DIRDEPDD ***********8777,32.50,credit,Paycheck,Checkarama,NaN,NaN
1,6/13/2022,CHASE CREDIT CRD AUTOPAY ***********2561,CHASE CREDIT CRD AUTOPAY ***********2561,271.89,debit,Credit Card Payment,Checkarama,NaN,NaN
2,6/13/2022,VENMO CASHOUT ***********7395,VENMO CASHOUT ***********7395,575.00,credit,Income,Checkarama,NaN,NaN
3,6/13/2022,PAYPAL INST XFER ***********DME*,PAYPAL INST XFER ***********DME*,25.00,debit,Transfer,Checkarama,NaN,NaN
4,6/13/2022,CASH APP*RICKY 8774174551 CA,CASH APP*RICKY 8774174551 CA,20.55,debit,Cash & ATM,Checkarama,NaN,NaN


In [7]:
transactions.tail(10)

,Date,Description,Original Description,Amount,Transaction Type,Category,Account Name,Labels,Notes
2260,5/04/2018,Parkplus Llc,"PARKPLUS, LLC",13.00,debit,Restaurants,CREDIT CARD,NaN,NaN
2261,5/03/2018,Starbucks,STARBUCKS STORE 08972 CHICAGO IL C_RESTAURANTS...,2.73,debit,Coffee Shops,Checkarama,NaN,NaN
2262,5/03/2018,Transfer to CREDIT CARD,CHASE CREDIT CRD AUTOPAY ***********0436 C_CRE...,2479.34,debit,Credit Card Payment,Checkarama,NaN,NaN
2263,5/03/2018,Presence Care Tr,PRESENCE CARE TR DIR DEP ***********7404 C_PAY...,2039.08,credit,Paycheck,Checkarama,NaN,NaN
2264,5/03/2018,Il Tollway Auto,IL TOLLWAY AUTO REPLENISH,20.00,debit,Public Transportation,CREDIT CARD,NaN,NaN
2265,5/03/2018,Exxon,EXXONMOBIL 97472625,25.16,debit,Gas & Fuel,CREDIT CARD,NaN,NaN
2266,5/02/2018,Transfer from Checkarama,AUTOMATIC PAYMENT - THANK,2479.34,credit,Credit Card Payment,CREDIT CARD,NaN,NaN
2267,5/01/2018,Aldi,ALDI 40065,63.55,debit,Groceries,CREDIT CARD,NaN,NaN
2268,5/01/2018,El Famous Burrito,EL FAMOUS BURRITO,11.55,debit,Restaurants,CREDIT CARD,NaN,NaN
2269,5/01/2018,Aldi,ALDI 40065,3.18,debit,Groceries,CREDIT CARD,NaN,NaN


In [5]:
transactions.shape

(2270, 9)

In [6]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2270 entries, 0 to 2269
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Date                  2270 non-null   object 
 1   Description           2270 non-null   object 
 2   Original Description  2270 non-null   object 
 3   Amount                2270 non-null   float64
 4   Transaction Type      2270 non-null   object 
 5   Category              2270 non-null   object 
 6   Account Name          2270 non-null   object 
 7   Labels                0 non-null      float64
 8   Notes                 0 non-null      float64
dtypes: float64(3), object(6)
memory usage: 159.7+ KB
